In [31]:
import os
import importlib.util
import pandas as pd
import yfinance as yf

In [32]:
def import_stocks(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File '{file_path}' not found.")
    spec = importlib.util.spec_from_file_location("stock_symbols", file_path)
    stock_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(stock_module)
    
    return stock_module.stock_symbols

In [33]:
def calculate_momentum(data, period):
    return data['Close'].rolling(window=period).sum()/period

In [34]:
def calculate_vwap_close(data):
    vwap = (data['Close'] * data['Volume']).cumsum() / data['Volume'].cumsum()
    vwap_close_ratio = vwap / data['Close']
    return vwap_close_ratio

In [35]:
def get_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data

In [36]:
def main():
    stock_symbols_file = "C:\\Users\\arind\\OneDrive\\Desktop\\stocks.py"  # Adjust the file path accordingly
    stock_symbols = import_stocks(stock_symbols_file)

    start_date = pd.Timestamp.today() - pd.DateOffset(days=180)
    end_date = pd.Timestamp.today()

    # Create a list to store common dates for all stocks
    common_dates = None

    filtered_symbols = []

    try:
        for symbol in stock_symbols:
            try:
                stock_data = get_stock_data(symbol, start_date, end_date)

                # Drop any rows with NaN values
                stock_data.dropna(inplace=True)

                if common_dates is None:
                    common_dates = set(stock_data.index)
                else:
                    common_dates = common_dates.intersection(set(stock_data.index))

            except Exception as e:
                print(f"Failed download: [{symbol}]: {e}")

        # Convert common_dates set to a sorted list
        common_dates = sorted(list(common_dates))

        for symbol in stock_symbols:
            try:
                stock_data = get_stock_data(symbol, start_date, end_date)

                # Filter the data to include only common dates for all stocks
                stock_data = stock_data.loc[common_dates]

                stock_data['1_week_momentum'] = calculate_momentum(stock_data, 5)
                stock_data['1_month_momentum'] = calculate_momentum(stock_data, 22)

                if len(stock_data) > 0:
                    filtered_stocks = stock_data[stock_data['1_week_momentum'] > stock_data['1_month_momentum']]

                    if len(filtered_stocks) > 0:
                        filtered_symbols.append(symbol)

                        vwap_close_ratio = calculate_vwap_close(filtered_stocks)

                        # Filter further based on VWAP/Close ratio condition (VWAP/Close > 1)
                        filtered_stocks = filtered_stocks[vwap_close_ratio > 1]
                        
                        if len(filtered_stocks) > 0:
                            print(f"Stock: {symbol}")
                            print(pd.DataFrame({'VWAP/Close Ratio': vwap_close_ratio}))
                            print("--------------------")

            except Exception as e:
                print(f"Failed download: [{symbol}]: {e}")

    except Exception as e:
        print("Error fetching or processing data:", e)

    print("Filtered Stock Symbols:", filtered_symbols)

In [37]:
if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [7]:
# Copy stock symbol list and feed into analysis tool